In [ ]:
shhh <- suppressPackageStartupMessages
#borrowed syntax from 
#[https://stackoverflow.com/questions/18931006/how-to-suppress-warning-messages-when-loading-a-library]

In [ ]:
shhh(library(dplyr))
shhh(library(tidyverse))
library(ggpubr)
library(ggrepel) 
library(ggplot2)
library(gganimate)
shhh(library(gridExtra))
shhh(library(magrittr))
shhh(library(lubridate))
library(tidyr)
shhh(library(plotly))
shhh(library(rstatix))
library(ggMarginal)

In [ ]:
#general aesthetic settings
th <- theme(plot.title = element_text(size = 30, face = 'bold'),
            axis.title = element_text(size = 20, face = 'bold'), 
            axis.text = element_text(size = 12), 
            strip.text = element_text(size = 12))

## General plotting functions

In [ ]:
#' state_reader function
#' a function that takes in 2 paths to dataframe and return a dataframe
#' @param state_temp_path: string object that is path of the temperature file at state level
#' @param cdc_path: string object that is path of the cdc provisional respiratory disease death count
#' @param name: string object that is name of the state
#' @return: a dataframe that is a merged table based on two dataframe provided above
#' @example: df <- state_reader('New York 2020-01-01 to 2022-04-01.csv', 
             #'Provisional_Death_Counts_for_Influenza__Pneumonia__and_COVID-19.csv', 
             #'New York')

state_reader <- function(state_temp_path, cdc_path, name){

    resp <- read.csv(cdc_path) #CDC
    temp <- read.csv(state_temp_path) #visual crossings

    mresp <- resp[,(grep("Deaths|Week.Ending.Date|Age.Group|Jurisdiction", 
                         colnames(resp)))]

    mresp <- mresp %>% 
        select(-c(Total.Deaths)) %>% 
        pivot_longer(cols = c(COVID.19.Deaths, 
                          Pneumonia.Deaths, 
                          Influenza.Deaths), 
                          names_to = 'Cause_of_death', 
                          values_to = 'Death_count') %>%
                          na.omit() %>% 
                          filter(Jurisdiction == name)
    
    mresp$Week.Ending.Date <- as.Date(mresp$Week.Ending.Date, format = '%m/%d/%Y')
    names(mresp)[1] <- c('datetime')
    temp$datetime <- as.Date(temp$datetime, method = 'ymd')
    state_resp_temp <- merge(mresp, temp)
    return(state_resp_temp)
}


In [ ]:
#' state_plotter function
#'
#' a function that takes in a dataframe, state name, and return a plot  
#' @param reader_output_table: a dataframe object that is generated by state_reader function
#' @param state: string object that is name of the state
#' @return: a dual y axis plot of temperature, death caused by respiratory diseases, and 
#' @example: state_plotter(test3, 'California')

state_plotter <- function(reader_output_table, state){
    coeff = 0.01
    options(repr.plot.width = 18, repr.plot.height=9)
    plot <- ggplot(reader_output_table, aes(x = datetime)) + 
            geom_point(aes(y = tempmax/coeff), 
                       color = '#EF8C71', 
                       alpha = 0.1 ) +
            geom_smooth(aes(y = tempmax/coeff), 
                        color = '#EF8C71', 
                        alpha = 0.1, 
                        span = 0.5, 
                        se = F)+

            geom_smooth( aes( y = tempmin/coeff), 
                        color = '#53B9BC', 
                        alpha = 0.1, 
                        span = 0.5, 
                        se = F) +
            geom_point( aes( y = tempmin/coeff), 
                       color = '#53B9BC', 
                       alpha = 0.1) +

            geom_point( aes( y = Death_count, color = Cause_of_death)) + 
            scale_y_continuous(
                name = 'Death count', 
                sec.axis = sec_axis(~.*coeff, name = 'Average temperature')) +
            labs(title = paste0(state, ' average maximum and minimum temperature\nand respiratory disease related death')) +
            xlab('Time from 2020 to 2022') + 
            theme_minimal() + 
            th + 
            theme(plot.title = element_text(size = 30, face = 'bold'), 
                  axis.title = element_text(size = 20, face = 'bold')) +
            facet_wrap(~Age.Group)
    
    return(plot)
    
}


In [1]:
#' state_plotter_ver2 function
#'
#' a function that takes in a dataframe, state name, and return a plot  
#' @param reader_output_table: a dataframe object that is generated by state_reader function
#' @param state: string object that is name of the state
#' @return: a dual y axis plot of temperature, death caused by respiratory diseases, and 
#' @example: state_plotter(test3, 'California')

state_plotter_ver2 <- function(reader_output_table, state){
    state_death_temp <- reader_output_table %>% filter(Age.Group == 'All Ages') %>%
    ggplot() + 
    aes( x = temp, y = Death_count, color = Cause_of_death) + 
    geom_point(size = 3) +
    labs(title = paste0(state, ' death count vs. temperature')) + 
    theme_bw() + 
    xlab('Temperature') + 
    ylab('Death count') + 
    th

    state_death_prec <- reader_output_table %>% filter(Age.Group == 'All Ages') %>%
    ggplot() + 
    aes( x = precip, y = Death_count, color = Cause_of_death, shape = preciptype) + 
    geom_point(size = 4) +
    labs(title = paste0(state, ' death count vs. precipitation')) +
    xlab('Precipitation') + 
    ylab('Death count') + 
    theme_bw() + 
    th

    options(repr.plot.width = 18, repr.plot.height=9)
    state_death_humidity <- reader_output_table %>% filter(Age.Group == 'All Ages') %>%
    ggplot() + 
    aes( x = humidity, y = Death_count, shape = preciptype, color = Cause_of_death) + 
    geom_point(size = 4) + 
    labs(title = paste0(state, ' death count vs. humidity')) + 
    theme_bw() + 
    xlab('Humidity') + 
    ylab('Death count') + 
    theme(plot.title = element_text(size = 30, face = 'bold'),
          axis.title = element_text(size = 20, face = 'bold'))
    
    p <- ggarrange(state_death_temp, state_death_prec, state_death_humidity)
    return(p)
}

In [ ]:
#' country_reader function
#'
#' a function that takes in a two file paths and return a dataframe  
#' @param country_path: a string object that is a file path to the Visual Crossing generated files
#' @param owid_path: a string object that is a file path to Our World in Data file
#' @param country_str: a string object that is the name of the country 
#' @return: a dataframe that is a merged table based on the inputs, for plotting 
#' @example: country_reader('')

country_reader <- function(country_path, owid_path, country_str){
    tab <- read.csv(owid_path)
    country_covid <- tab %>% filter(location == country_str)
    country <- read_csv(country_path)
    names(country_covid)[4] <- c('datetime')
    country_temp_covid <- merge(country, country_covid)
    country_temp_covid <- country_temp_covid[, !duplicated(colnames(country_temp_covid))]
    return(country_temp_covid)
}


#' country_plotter function
#'
#' a function that takes input dataframe and generated a plot
#' @param reader_output_table: a dataframe object generated by country_reader 
#' @param country_str: a string object for country name
#' @return a plot of weather factors (humidity, temperature, and precipitation) smoothed covid deaths
country_plotter <- function(reader_output_table, country_str){
        options(repr.plot.width = 8, repr.plot.height=8)
        temp_country_p <- ggplot(reader_output_table)+ 
                        aes(y = new_deaths_smoothed, 
                        x = temp) + 
                        geom_point(color = '#67C293') + 
                        geom_smooth(method = 'lm') + 
                        theme_bw() +
                        xlab('Temperature') +
                        ylab('New deaths smoothed') + 
                        th +  
        stat_cor(aes(label= paste(..rr.label.., sep = "~`,`~")),label.x = 3) +
        theme(plot.title = element_text(size = 16, face = 'bold'),
              axis.title = element_text(size = 16, face = 'bold')) + 
              labs(title = paste0(country_str, ' COVID deaths'))

        options(repr.plot.width = 8, repr.plot.height=8)
        humidity_country_p <- ggplot(reader_output_table)+ 
                        aes(y = new_deaths_smoothed, 
                        x = humidity) + 
                        geom_point(color = '#67C293') +  
                        geom_smooth(method = 'lm') + 
                        theme_bw() + 
                        xlab('Humidity') +
                        ylab('New deaths smoothed') + 
                        th +  
                        labs(title = '') + 
                        theme(plot.title = element_text(size = 20, face = 'bold'),
                              axis.title = element_text(size = 16, face = 'bold')) + 
        stat_cor(aes(label= paste(..rr.label.., sep = "~`,`~")),label.x = 3) 

        options(repr.plot.width = 8, repr.plot.height=8)
        precipitation_country_p <- ggplot(reader_output_table) + 
                        aes(y = new_deaths_smoothed, 
                        x = precip) + 
                        geom_point(color = '#67C293') + 
                        geom_smooth(method = 'lm') + 
                        xlab('Precipitation') + 
                        ylab('New deaths smoothed') + 
                        theme_bw() + 
                        th + 
                        theme(plot.title = element_text(size = 20, face = 'bold'),
                              axis.title = element_text(size = 16, face = 'bold')) + 
                        #labs(title ='country COVID deaths\nvs. precipitation') +
        stat_cor(aes(label= paste(..rr.label.., sep = "~`,`~")),label.x = 3) 

        plot <- ggarrange(temp_country_p, humidity_country_p, precipitation_country_p, 
                  ncol = 2, nrow = 2)
    return(plot)
    }



**Plotting functions to look at individual waves of COVID spikes**

In [ ]:
# Create additional dataframes by filtering values based on the above information

# Create a function
# Takes in range of dates in the format of a list
# Return a list of dataframes, each of which is one wave in the list

wave_reader <- function(inputvec, dataframe){
    #number of wave can be determined by number of elements in list
    #make sure that number of elements in list is even,
    anslist <- list()
    if (length(inputvec) %% 2 != 0){
        print ('Please make sure there are even numbers of dates in the list')
        return (0)
    }
    vec_wave <- paste0('Wave ', seq(length(inputvec)/2)) #vector of wave
    #print(vec_wave)
    for ( i in 1:length(vec_wave)){  #iterate over all elements of the vector
         
        start_date <- inputvec[i*2 - 1]
        end_date <- inputvec[i*2]
        #print(paste0(start_date, end_date))
        wave <- dataframe %>% filter(between(datetime, as.Date(start_date), as.Date(end_date)))
        anslist <- c(anslist, list(wave))
    }   
    return(anslist)
}

#take in input form the above and generate a list of plots
wave_plotter <- function(inputlist, name){
    plot_list <- list()
    for (i in 1:length(inputlist)){
        plot <- plotter_ver2(as.data.frame(inputlist[i]), paste0('Wave ', i, '\n', name))
        show(plot)
        options(repr.plot.width = 18, repr.plot.height=5)
        plot_list <- append(plot_list, list(plot))
    }
    p <- ggarrange(plotlist = plot_list, 
                  ncol = 1, 
                  nrow = length(inputlist))
    return(p)
}

Eyeballing and plotting the different COVID spikes <br>
If two spikes (local max) overlap closely, they are counted as one wave for simplicity <br>
Each states being considered has 3 - 4 waves 

In [ ]:


#Roughly assign waves based on local mins smoothed death count
#Eye-balling 

#California
#wave 1: 2020-03-21 to 2020-10-24
#wave 2: 2020-10-24 to 2021-04-10
#wave 3: 2021-07-24 to 2022-03-19

Cali <- state_reader('California 2020-01-01 to 2022-04-01.csv',
             'Provisional_Death_Counts_for_Influenza__Pneumonia__and_COVID-19.csv', 
             'California')

sample <- wave_reader(c('2020-03-21', '2020-10-24', 
                        '2020-10-24', '2021-04-10', 
                        '2021-07-24', '2022-03-19'), Cali)

ggsave('Plot_for_final/Waves.Cali.jpg', width=18, height=16, wave_plotter(sample, 'California'))

#New York
#wave 1: 2020-03-21 to 2020-07-11
#wave 2: 2020-11-07 to 2021-06-05
#wave 3: 2021-08-07 to 2022-03-26

NY <- state_reader('New York 2020-01-01 to 2022-04-01.csv',
             'Provisional_Death_Counts_for_Influenza__Pneumonia__and_COVID-19.csv', 
             'New York')

sample <- wave_reader(c('2020-03-21', '2020-07-11', 
                        '2020-11-07', '2021-06-05', 
                        '2021-08-07', '2022-03-26'), NY)

ggsave('Plot_for_final/Waves.NewYork.jpg', width=18, height=16, wave_plotter(sample, 'New York'))

#Texas
#wave 1: 2020-03-28 to 2020-10-03
#wave 2: 2020-10-10 to 2021-06-05
#wave 3: 2021-07-17 to 2021-11-20
#wave 4: 2021-12-25 to 2022-04-23

Texas <- state_reader('Temperature_Texas_2020-01-01_to_2022-08-01.csv',
             'Provisional_Death_Counts_for_Influenza__Pneumonia__and_COVID-19.csv', 
             'Texas')

sample <- wave_reader(c('2020-03-28', '2020-10-03', 
                        '2020-10-10', '2021-06-05',
                        '2021-07-17', '2021-11-20',
                        '2021-12-25', '2022-04-23'), Texas)
ggsave('Plot_for_final/Waves.Texas.jpg', width=18, height=16, wave_plotter(sample, 'Texas'))

#Michigan
#wave 1: 2020-03-21 to 2020-06-27
#wave 2: 2020-10-10 to 2021-03-13
#wave 3: 2021-03-20 to 2021-07-03
#wave 4: 2021-07-31 to 2022-03-26

Michigan <- state_reader('Michigan 2020-01-01 to 2022-04-01.csv',
             'Provisional_Death_Counts_for_Influenza__Pneumonia__and_COVID-19.csv', 
             'Michigan')

sample <- wave_reader(c('2020-03-21', '2020-06-27', 
                        '2020-10-10', '2021-03-13', 
                        '2021-03-20', '2021-07-03', 
                        '2021-07-31', '2022-03-26' ), Michigan)
ggsave('Plot_for_final/Waves.Michigan.jpg', width=18, height=16, wave_plotter(sample, 'Michigan'))


#Delaware
#wave 1: 2020-04-11 to 2020-07-25
#wave 2: 2020-10-17 to 2021-05-08
#wave 3: 2021-08-21 to 2022-03-26
Delaware <- state_reader('Delaware 2020-01-01 to 2022-04-01.csv',
             'Provisional_Death_Counts_for_Influenza__Pneumonia__and_COVID-19.csv', 
             'Delaware')

sample <- wave_reader(c('2020-04-11', '2020-07-25', 
                        '2020-10-17', '2021-05-08', 
                        '2021-08-21', '2022-03-26'), Delaware)

ggsave('Plot_for_final/Waves.Delaware.jpg', width=18, height=16, wave_plotter(sample, 'Delaware'))



## Plotting scripts

#### COVID deaths and fraction of elderly population

In [ ]:

options(repr.plot.width = 13, repr.plot.height=10)
mort_all[!duplicated(mort_all$location),] %>%
    ggplot() + 
    aes(y = total_deaths_per_million, x = elder, label = location) + 
    geom_point(color = '#74CBF6') + geom_smooth( method = 'lm', color = '#255E9F') + 
    theme_minimal() + 
    stat_cor(aes(label = paste(..rr.label.., ..p.label.., sep = "~`,`~")),label.x = 3) +
    geom_label_repel(aes(label = location)) +
    ylab('Total deaths') + 
    xlab('Share of elder population') + 
    th + 
    labs(title = c('COVID total deaths per million and\nfraction elder 2020-2022') 
        ) 

ggsave('Plot_for_final/Fraction_elder_total_deaths.png', width = 13, height = 10)

#### COVID deaths/cases and wealth

**A. Dataframes**

In [ ]:
tab <- read.csv('owid-covid-data.csv')

In [ ]:
mort_all <- tab %>% select(mortality_rate = new_deaths_per_million, 
               location, continent, 
               total_cases_per_million, 
               population,
               elder = aged_65_older,
               total_deaths_per_million,
               hosp = weekly_hosp_admissions_per_million,
               gdp_per_capita, extreme_poverty, 
               new_tests_smoothed_per_thousand, 
               date)

In [4]:
# Creating bins of GDP per capita
small_mort <- mort_all[!duplicated(mort_all$location),] %>%
mutate(gdp_bins = as.factor(ntile(gdp_per_capita, 3))) %>%
filter(!is.na(gdp_bins)) 

ERROR: Error in mort_all[!duplicated(mort_all$location), ] %>% mutate(gdp_bins = as.factor(ntile(gdp_per_capita, : could not find function "%>%"


**B1. Scatter plots**

**Covid and GDP**

In [ ]:
#COVID cases and GDP
options(repr.plot.width = 10, repr.plot.height=10)
case_gdp <- mort_all[!duplicated(mort_all$location),] %>%
    ggplot() + 
    aes(y = total_cases_per_million, x = gdp_per_capita, label = location, color = continent) + 
    geom_point() + 
    theme_minimal() + 
    #geom_label_repel(aes(label = location)) +
    ylab('Total cases per million') + 
    xlab('GDP per capita') + 
    labs(title = c('COVID total cases per million and GDP per capita 2020-2022')) 

show(case_gdp)
ggplotly(case_gdp)

**COVID and extreme poverty**

In [ ]:
options(repr.plot.width = 12, repr.plot.height=10)
deaths_poverty <- mort_all[!duplicated(mort_all$location),] %>%
    ggplot() + 
    aes(y = total_deaths_per_million, x = extreme_poverty, label = location, color = gdp_per_capita) + 
    geom_point() + 
    #geom_smooth( method = 'lm', color = '#255E9F') + no point in plotting a line where there isn't one to be plotted
    theme_minimal() + 
    scale_color_viridis_c() +
    #stat_cor(aes(label = paste(..rr.label.., ..p.label.., sep = "~`,`~")), label.x = 6) +
    #geom_label_repel(aes(label = location)) +
    ylab('Total deaths per million') +
    xlab('Extreme poverty') + 
    labs(title = c('COVID total deaths per million and \nfraction of population that lives in poverty 2020-2022') 
        ) 
ggplotly(deaths_poverty)
show(deaths_poverty)

In [ ]:
options(repr.plot.width = 15, repr.plot.height=10)
cases_poverty <- mort_all[!duplicated(mort_all$location),] %>%
    ggplot() + 
    aes(y = total_cases_per_million, x = extreme_poverty, label = location, color = gdp_per_capita) + 
    geom_point() + 
    #geom_smooth( method = 'lm', color = '#255E9F') + no point in plotting a line where there isn't one to be plotted
    theme_minimal() + 
    scale_color_viridis_c() +
    #stat_cor(aes(label = paste(..rr.label.., ..p.label.., sep = "~`,`~")), label.x = 6) +
    geom_label_repel(aes(label = location)) +
    ylab('Total cases per million') +
    xlab('Extreme poverty') + 
    labs(title = c('COVID total cases per million and \nfraction of population that lives in poverty 2020-2022') 
        ) 
ggplotly(cases_poverty)
show(cases_poverty + th)

**Three bins of GDP**

In [ ]:
#creating three bins of GDP, and only looking at the first bin,

#applying data filters, 
#first, removing all duplicated entries
#then, generating a gdp bins
#next, remove all countries where there are no data on GDP


options(repr.plot.width = 15, repr.plot.height=12)
levels(small_mort$gdp_bins) <- c('Lower GDP bin', 'Middle GDP bin', 'Upper GDP bin') 
    ggplot(small_mort) + 
    aes(y = total_deaths_per_million, x = gdp_per_capita, label = location, color = gdp_bins) + 
    geom_point() + geom_smooth( method = 'lm', color = '#255E9F') + 
    theme_minimal() + 
    stat_cor(aes(label = paste(..rr.label.., ..p.label.., sep = "~`,`~")), label.x = 6) +
    geom_label_repel(aes(label = location)) +
    ylab('Total deaths per million') + 
    xlab('GDP per capita') +
    theme(legend.position = 'none', strip.text = element_text(size = 16, face = 'bold')) +
    th +
    facet_wrap(~gdp_bins, ncol=2) +
    labs(title = c('COVID total deaths per million 2020-2022 and \ngdp per capita') 
        ) 
ggsave('Plot_for_final/gdp_mort_labeled.png', width = 15, height = 12)

#### B2. Boxplots <br>
**Box plots for mortality across different GDP bins** 

In [ ]:
#boxplot of mortality data based
comparison <- list(c('Lower GDP bin', 'Middle GDP bin'), 
                   c('Middle GDP bin', 'Upper GDP bin'), 
                   c('Upper GDP bin', 'Lower GDP bin'))
    

options(repr.plot.width = 8, repr.plot.height=8)

    ggplot(small_mort) + 
    aes(y = total_deaths_per_million, x = gdp_bins, color = gdp_bins) + 
    geom_boxplot() + 
    theme_minimal() + 
    ylab('Total deaths per million') + 
    xlab('GDP per capita') +
    theme(legend.position = 'none', strip.text = element_text(size = 16, face = 'bold')) +
    th +
    ggpubr::stat_compare_means(comparisons = comparison, label = 'p.signif') +
    labs(title = c('COVID total deaths per million 2020-2022 and \ngdp per capita') 
        ) 
ggsave('Plot_for_final/boxplot_covid_death_gdp_bins.png', width = 8, height = 8)

**Boxplot functions for COVID deaths and temperature across different waves**

In [3]:
#create a function that makes boxplots of daily death counts for each state
#takes in a list and return a dataframe
#inputlist is generated by function wave_reader from above section

boxplot_reader <- function(inputlist){
    findf <- data.frame()
    for (i in 1:length(inputlist)){
        df <- as.data.frame(inputlist[i])
        df <- df %>% mutate(Wave = rep(i, nrow(df)))
        findf <- rbind(findf, df)
    }
    findf$Wave <- as.factor(findf$Wave)
    levels(findf$Wave) <- c('Wave 1', 'Wave 2', 'Wave 3')
    return(findf)
}
    


In [ ]:
#ascending order from left to right based on mean temperature during the wave

boxplot_plotter <- function(dataframe, state){
    options(repr.plot.width = 10, repr.plot.height = 8)
        
        if (length(unique(dataframe$Wave)) == 3){
            comparison <- list(c('Wave 1', 'Wave 2'), 
                               c('Wave 1', 'Wave 3'), 
                               c('Wave 2', 'Wave 3'))
            }
        else {
            comparison <- list(c('Wave 1', 'Wave 2'), 
                               c('Wave 1', 'Wave 3'), 
                               c('Wave 1', 'Wave 4'), 
                               c('Wave 2', 'Wave 3'), 
                               c('Wave 2', 'Wave 4'), 
                               c('Wave 3', 'Wave 4'))
        }
 
    dataframe %>% filter(Cause_of_death == 'COVID.19.Deaths') %>% 
             filter(Age.Group == 'All Ages') %>%
             mutate(fact = fct_reorder(Wave, temp, .fun = 'mean')) %>%
    ggplot() + 
    aes(x=reorder(fact, temp), y = Death_count, fill = Wave) + 
    geom_boxplot() + 
    ggpubr::stat_compare_means(comparisons = comparison, label = 'p.signif') +
    theme_bw() + 
    scale_fill_brewer(palette = 'Set1') +
    labs(title = paste0('Daily COVID death counts\nacross different waves in ', state)) +
    theme(plot.title = element_text(size = 20, face = 'bold'),
         legend.position = 'none', 
         axis.text = element_text(size = 16), 
         axis.title = element_text(size = 18, face = 'bold'), 
         panel.grid = element_blank()) + 
    xlab('Waves of major COVID spikes') +
    ylab('Smoothed daily death counts')
    
}

In [ ]:
#also boxplots, but this time for temperature instead of Daily death counts
boxplot_plotter_2 <- function(dataframe, state){
    options(repr.plot.width = 10, repr.plot.height = 8)
        if (length(unique(dataframe$Wave)) == 3){
            comparison <- list(c('Wave 1', 'Wave 2'), 
                               c('Wave 1', 'Wave 3'), 
                               c('Wave 2', 'Wave 3'))
            }
        
        else {
            comparison <- list(c('Wave 1', 'Wave 2'), 
                               c('Wave 1', 'Wave 3'), 
                               c('Wave 1', 'Wave 4'), 
                               c('Wave 2', 'Wave 3'), 
                               c('Wave 2', 'Wave 4'), 
                               c('Wave 3', 'Wave 4'))
        }
    dataframe %>% filter(Cause_of_death == 'COVID.19.Deaths') %>% 
             filter(Age.Group == 'All Ages') %>%
             mutate(fact = fct_reorder(Wave, temp, .fun = 'mean')) %>%
    ggplot() + 
    aes(x=reorder(fact, temp), y = temp, fill = Wave) + 
    geom_boxplot() + 
    ggpubr::stat_compare_means(comparisons = comparison, label = 'p.signif') +
    theme_bw() + 
    scale_fill_brewer(palette = 'Set1') +
    labs(title = paste0('Average temperature\nacross different COVID waves in ', state)) +
    theme(plot.title = element_text(size = 20, face = 'bold'),
         legend.position = 'none', 
         axis.text = element_text(size = 16), 
         axis.title = element_text(size = 18, face = 'bold'), 
         panel.grid = element_blank()) + 
    xlab('Waves of major COVID spikes') +
    ylab('Average daily temperature')
    
}

**Exec scripts for the boxplot functions**

In [ ]:
#helper functions that chain call other functionse
func_caller <- function(state_temp, CDC_resp, state_name, wave_vec){
    State <- state_reader(state_temp, CDC_resp, state_name) # Create an initial dataframe from raws 
    sample <- wave_reader(wave_vec, State)                  # Generating a list of different waves
    test <- boxplot_reader(sample)                          # Create a dataframe for box-plotting
    p1 <- boxplot_plotter(test, state_name) # daily deaths vs. waves
    p2 <-boxplot_plotter_2(test, state_name) # daily temperature vs waves
    return(ggarrange(p1, p2)
}


In [ ]:
CDC <- 'Provisional_Death_Counts_for_Influenza__Pneumonia__and_COVID-19.csv'
Cali_wave_vec <- c('2020-03-21', '2020-10-24', '2020-10-24', '2021-04-10', '2021-07-24', '2022-03-19')
Cali <- 'California 2020-01-01 to 2022-04-01.csv'

func_caller(Cali, CDC, 'California', Cali_wave_vec)
ggsave('Plot_for_final/2_boxplots_waves_Cal.png', width = 14, height = 8)

NY <- 'New York 2020-01-01 to 2022-04-01.csv'
NY_wave_vec <- c('2020-03-21', '2020-07-11', 
                 '2020-11-07', '2021-06-05', 
                 '2021-08-07', '2022-03-26')
func_caller(NY, CDC, 'New York', NY_wave_vec)
ggsave('Plot_for_final/2_boxplots_waves_NY.png', width = 14, height = 8)

TX <- 'Temperature_Texas_2020-01-01_to_2022-08-01.csv'
TX_wave_vec <- c('2020-03-28', '2020-10-03', 
                 '2020-10-10', '2021-06-05',
                 '2021-07-17', '2021-11-20',
                 '2021-12-25', '2022-04-23')
func_caller(TX, CDC, 'Texas', TX_wave_vec)
ggsave('Plot_for_final/2_boxplots_waves_TX.png', width = 14, height = 8)

Mi <- 'Michigan 2020-01-01 to 2022-04-01.csv'
Mi_wave_vec <- c('2020-03-21', '2020-06-27', 
                '2020-10-10', '2021-03-13', 
                '2021-03-20', '2021-07-03', 
                '2021-07-31', '2022-03-26' )
func_caller(Mi, CDC, 'Michigan', Mi_wave_vec)
ggsave('Plot_for_final/2_boxplots_waves_Mi.png', width = 14, height = 8)

DE <- 'Delaware 2020-01-01 to 2022-04-01.csv'
DE_wave_vec <- c('2020-04-11', '2020-07-25', 
                 '2020-10-17', '2021-05-08', 
                 '2021-08-21', '2022-03-26')
func_caller(DE, CDC, 'Delaware', DE_wave_vec)
ggsave('Plot_for_final/2_boxplots_waves_DE.png', width = 14, height = 8)

#### B3. Mean_SE plot
**Deaths across three bins of GDP**

In [ ]:
#calculating mean and sd of deaths across three categories

mean_sd <- small_mort %>% 
           group_by(gdp_bins) %>% 
           summarize(
               mean_total_deaths = mean(total_deaths_per_million, na.rm = TRUE), 
               sd_total_deaths = sd(total_deaths_per_million, na.rm = TRUE)
           )

count_data <- count(small_mort, gdp_bins)
mean_sd <- merge(mean_sd, count_data) %>% mutate(se_total_deaths = sd_total_deaths / sqrt(n))


options(repr.plot.width = 8, repr.plot.height=8)
ggplot(mean_sd) +
    aes(x=gdp_bins, y = mean_total_deaths) +
    geom_errorbar(aes(ymin=mean_total_deaths - se_total_deaths, 
                      ymax=mean_total_deaths + se_total_deaths, width=.1)) + 
    theme_minimal() + 
    geom_point() + 
    th + 
    labs(title = c('Mean +/- SE total deaths based on \nthree bins of GDP'))
ggsave('Plot_for_final/Mean_se_gdp_bins.png')

#### B4. Scatter plot with histogram margins

In [ ]:
#people vaccinated and death counts
options(repr.plot.width=8, repr.plot.height=8)
p <- tab %>% 
filter(location == 'World') %>%
    filter(!is.na(people_fully_vaccinated_per_hundred)) %>%
    ggplot() + 
    aes(y = new_deaths_smoothed, x = people_fully_vaccinated_per_hundred) + 
    geom_point(color = '#74CBF6') + geom_smooth( method = 'lm', color = '#255E9F') + 
    theme_minimal() + 
    stat_cor(aes(label = paste(..rr.label.., ..p.label.., sep = "~`,`~")),label.x = 3) + 
    xlab('People fully vaccinated') + 
    ylab('New deaths smoothed') +
    #facet_wrap(~location, ncol = 9) +
    theme(plot.title = element_text(size = 16)) + 
    labs(title = c ('World level\nCOVID smoothed new deaths vs. people fully vaccinated')) 

p2 <- ggMarginal(p, type = 'histogram', fill = '#74CBF6', color = '#255E9F')


**C. Table writer** <br>
**Mean and sd of average death counts**

In [ ]:

#dataframe is input generated by 
Wave_table_writer <- function(dataframe, state) {
    smalltest <- dataframe %>% filter(Cause_of_death == 'COVID.19.Deaths') %>% 
             filter(Age.Group == 'All Ages') %>% 
             group_by(Wave) 

    df1 <- smalltest %>% summarize(Mean_daily_death = mean(Death_count), 
                                   SD_daily_death = sd(Death_count), 
                                   Mean_temp = mean(temp), 
                                   SD_temperature = sd(temp))
    df2 <- count(smalltest)
    df3 <- merge(df1, df2)
    write.table(paste0('Data_for_final/COVID_waves_deaths_temperature_', state), df3)
}